In [1]:
# Create Engine

from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

engine = create_engine('sqlite:///:memory:')

Session = sessionmaker(bind=engine)

session = Session()


In [2]:
# Create tables

from datetime import datetime

from sqlalchemy import (Table, Column, Integer, Numeric, String, DateTime,
                        ForeignKey, Boolean, CheckConstraint)
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, backref


Base = declarative_base()


class Cookie(Base):
    __tablename__ = 'cookies'
    __table_args__ = (CheckConstraint(
            'quantity >= 0', name='quantity_positive'),)

    cookie_id = Column(Integer, primary_key=True)
    cookie_name = Column(String(50), index=True)
    cookie_recipe_url = Column(String(255))
    cookie_sku = Column(String(55))
    quantity = Column(Integer())
    unit_cost = Column(Numeric(12, 2))

    def __init__(self, name, recipe_url=None, sku=None, 
                 quantity=0, unit_cost=0.00):
        self.cookie_name = name
        self.cookie_recipe_url = recipe_url
        self.cookie_sku = sku
        self.quantity = quantity
        self.unit_cost = unit_cost

    def __repr__(self):
        return "Cookie(cookie_name='{self.cookie_name}', " \
                       "cookie_recipe_url='{self.cookie_recipe_url}', " \
                       "cookie_sku='{self.cookie_sku}', " \
                       "quantity={self.quantity}, " \
                       "unit_cost={self.unit_cost})".format(self=self)


class User(Base):
    __tablename__ = 'users'

    user_id = Column(Integer(), primary_key=True)
    username = Column(String(15), nullable=False, unique=True)
    email_address = Column(String(255), nullable=False)
    phone = Column(String(20), nullable=False)
    password = Column(String(25), nullable=False)
    created_on = Column(DateTime(), default=datetime.now)
    updated_on = Column(DateTime(), default=datetime.now, onupdate=datetime.now)

    def __init__(self, username, email_address, phone, password):
        self.username = username
        self.email_address = email_address
        self.phone = phone
        self.password = password

    def __repr__(self):
        return "User(username='{self.username}', " \
                     "email_address='{self.email_address}', " \
                     "phone='{self.phone}', " \
                     "password='{self.password}')".format(self=self)


class Order(Base):
    __tablename__ = 'orders'
    order_id = Column(Integer(), primary_key=True)
    user_id = Column(Integer(), ForeignKey('users.user_id'))
    shipped = Column(Boolean(), default=False)

    user =  relationship("User", backref=backref('orders', order_by=order_id))

    def __repr__(self):
        return "Order(user_id={self.user_id}, " \
                      "shipped={self.shipped})".format(self=self)


class LineItem(Base):
    __tablename__ = 'line_items'
    line_item_id = Column(Integer(), primary_key=True)
    order_id = Column(Integer(), ForeignKey('orders.order_id'))
    cookie_id = Column(Integer(), ForeignKey('cookies.cookie_id'))
    quantity = Column(Integer())
    extended_cost = Column(Numeric(12, 2))

    order = relationship("Order", backref=backref('line_items',
                                                  order_by=line_item_id))
    cookie = relationship("Cookie", uselist=False)

    def __repr__(self):
        return "LineItems(order_id={self.order_id}, " \
                          "cookie_id={self.cookie_id}, " \
                          "quantity={self.quantity}, " \
                          "extended_cost={self.extended_cost})".format(
                    self=self)

Base.metadata.create_all(engine)

In [3]:
# Create instance of a cookie

cc_cookie = Cookie('chocolate chip',
                  'http://some.chocolate.chip.com',
                  'CC01',
                  12,
                  0.50)


In [4]:
# Inspect instance

from sqlalchemy import inspect
insp = inspect(cc_cookie)


In [5]:
# Get the session state of an instance
for state in ['transient', 'pending', 'persistent', 'detached']:
    print('{:>10}: {}'.format(state, getattr(insp, state)))

 transient: True
   pending: False
persistent: False
  detached: False


In [6]:
# Add cookie
session.add(cc_cookie)

In [7]:
# Inspect again
for state in ['transient', 'pending', 'persistent', 'detached']:
    print('{:>10}: {}'.format(state, getattr(insp, state)))

 transient: False
   pending: True
persistent: False
  detached: False


In [8]:
# Modify cookie_name attribute
cc_cookie.cookie_name = 'Change chocolate chip'
insp.modified

True

In [9]:
# Print the changed attribute history
for attr, attr_state in insp.attrs.items():
    if attr_state.history.has_changes():
        print('{}: {}'.format(attr, attr_state.value))
        print('HIstory: {}\n'.format(attr_state.history))

unit_cost: 0.5
HIstory: History(added=[0.5], unchanged=(), deleted=())

cookie_sku: CC01
HIstory: History(added=['CC01'], unchanged=(), deleted=())

cookie_name: Change chocolate chip
HIstory: History(added=['Change chocolate chip'], unchanged=(), deleted=())

cookie_recipe_url: http://some.chocolate.chip.com
HIstory: History(added=['http://some.chocolate.chip.com'], unchanged=(), deleted=())

quantity: 12
HIstory: History(added=[12], unchanged=(), deleted=())



In [10]:
cc_cookie.quantity

12

In [11]:
# Add cookiemon

cookiemon = User('cookiemon', 
                 'mon@cookie.com', 
                 '111-111-1111',
                 'password')

session.add(cookiemon)

In [12]:
# Add dark chocolate chip

dcc_cookie = Cookie('dark choclate chip',
                   'http://something.something.com',
                   'CC02', 
                   1,
                   0.75)

session.add(dcc_cookie)

In [13]:
# Add first order

o1 = Order()
o1.user = cookiemon
session.add(o1)

line1 = LineItem(
    order=o1,
    cookie=cc_cookie,
    quantity=9,
    extended_cost=4.50
)

session.add(line1)
session.commit()


In [14]:
# Add second order

o2 = Order()
o2.user = cookiemon
session.add(o2)

line1 = LineItem(
    order=o2,
    cookie=cc_cookie,
    quantity=2,
    extended_cost=1.50
)

line2 = LineItem(
    order=o2,
    cookie=dcc_cookie,
    quantity=9,
    extended_cost=6.75
)

session.add(line1)
session.add(line2)
session.commit()


/Users/rodrigovillatoro/venv/lib/python3.5/site-packages/sqlalchemy/sql/sqltypes.py:562: SAWarning: Dialect sqlite+pysqlite does *not* support Decimal objects natively, and SQLAlchemy must convert from floating point - rounding errors and other issues may occur. Please consider storing Decimal numbers as strings or integers on this platform for lossless storage.
  'storage.' % (dialect.name, dialect.driver))


In [15]:
# Just checking for current quantities

order2 = session.query(Order).get(2)
for line_item in order2.line_items:
    print(line_item.cookie)
    print(line_item.cookie.quantity)

Cookie(cookie_name='Change chocolate chip', cookie_recipe_url='http://some.chocolate.chip.com', cookie_sku='CC01', quantity=12, unit_cost=0.50)
12
Cookie(cookie_name='dark choclate chip', cookie_recipe_url='http://something.something.com', cookie_sku='CC02', quantity=1, unit_cost=0.75)
1


/Users/rodrigovillatoro/venv/lib/python3.5/site-packages/sqlalchemy/sql/sqltypes.py:562: SAWarning: Dialect sqlite+pysqlite does *not* support Decimal objects natively, and SQLAlchemy must convert from floating point - rounding errors and other issues may occur. Please consider storing Decimal numbers as strings or integers on this platform for lossless storage.
  'storage.' % (dialect.name, dialect.driver))


In [16]:
# Defining ship_it function

def ship_it(order_id):
    order = session.query(Order).get(order_id)
    for li in order.line_items:
        li.cookie.quantity = li.cookie.quantity - li.quantity
        session.add(li.cookie)
    order.shipped = True
    session.add(order)
    session.commit()
    print('shipped order ID: {}'.format(order_id))

In [17]:
# Ship the first order
ship_it(1)
print(session.query(Cookie.cookie_name, Cookie.quantity).all())

shipped order ID: 1
[('Change chocolate chip', 3), ('dark choclate chip', 1)]


In [18]:
# Ship second order ... AND GET AN INTEGRITYERROR
ship_it(2)

IntegrityError: (sqlite3.IntegrityError) CHECK constraint failed: quantity_positive [SQL: 'UPDATE cookies SET quantity=? WHERE cookies.cookie_id = ?'] [parameters: ((1, 1), (-8, 2))]

In [19]:
# To recover from error, we would need to manualy 
# issue a rollback(), like so:

session.rollback()
print(session.query(Cookie.cookie_name, Cookie.quantity).all())

[('Change chocolate chip', 3), ('dark choclate chip', 1)]


In [20]:
# Transactional ship_it: we first need to modify the ship_it
# function to guard agains IntegrityError. rolback() if error.

from sqlalchemy.exc import IntegrityError


def ship_it(order_id):
    order = session.query(Order).get(order_id)
    for li in order.line_items:
        li.cookie.quantity = li.cookie.quantity - li.quantity
        session.add(li.cookie)
    order.shipped = True
    session.add(order)
    try:
        session.commit()
        print('shipped order ID: {}'.format(order_id))
    except IntegrityError as error:
        print('ERROR: {!s}'.format(error.orig))
        session.rollback()

In [21]:
# Try shipping order 2 again... AND NOW GET PROPER ERROR

ship_it(2)

ERROR: CHECK constraint failed: quantity_positive


In [22]:
# Just checking inventory... AND EVERYTHING IS OK

print(session.query(Cookie.cookie_name, Cookie.quantity).all())

[('Change chocolate chip', 3), ('dark choclate chip', 1)]
